In [3]:
import nibabel as nib
import numpy as np
from scipy import stats
import mvlearn
from mvlearn import embed
import pandas as pd
import os
import re
import subprocess as sp
import glob
import hcp_utils as hcp
import ciftools_af as ct


### Smooth time series using wb_command
def smooth_dtseries(in_dtseries, out_dtseries, kernel, L_surface, R_surface):
    sp.run(f'wb_command -cifti-smoothing {in_dtseries} -fwhm {kernel} {kernel} \
    COLUMN {out_dtseries} -left-surface {L_surface} -right-surface {R_surface}',shell=True)
    

### Load the time series of the L and R hemisphere 
def load_dtseries(dtseries_path):
    # load the file
    dtseries = nib.load(dtseries_path)
    # find the offset and number of vertices for each hemisphere
    L_model = [x for x in dtseries.header.get_index_map(1).brain_models if x.brain_structure=='CIFTI_STRUCTURE_CORTEX_LEFT']
    R_model = [x for x in dtseries.header.get_index_map(1).brain_models if x.brain_structure=='CIFTI_STRUCTURE_CORTEX_RIGHT']
    offset_count = [L_model[0].index_offset, L_model[0].index_count,
                    R_model[0].index_offset, R_model[0].index_count]
    # extract the cortical timeseries
    values = dtseries.get_fdata()
    values = values[0:,np.append(np.arange(offset_count[0],offset_count[0]+offset_count[1]),np.arange(offset_count[2],offset_count[2]+offset_count[3]))]
    
    return values


### Z-score and conatenation of dtseries of all subject folders in a directory
def concat_dtseries(paths_to_dtseries):
    if len(paths_to_dtseries)<2:
        print(f'At least 2 time series are needed: {len(paths_to_dtseries)} found')
    dtseries_lst = [load_dtseries(path) for path in paths_to_dtseries]
    dtseries_lst = [hcp.normalize(dtseries) for dtseries in dtseries_lst]
        
    return np.concatenate(dtseries_lst)


### Store gradients in cifti2 dscalar.nii
def mk_grad1_dscalar(grads, template_cifti, output_dir):
    # grads: array with dimensions gradients X vertices
    # template_cifti: any cifti2 file with a BrainModelAxis (I am using one of the dtseries.nii)
    # output_dir: path to output directory
    data = np.zeros([grads.shape[0],template_cifti.shape[1]])
    data[0:,0:grads.shape[1]] = grads

    map_labels = [f'Measure {i+1}' for i in range(grads.shape[0])]
    ax0 = nib.cifti2.cifti2_axes.ScalarAxis(map_labels)
    ax1 = nib.cifti2.cifti2_axes.from_index_mapping(template_cifti.header.get_index_map(1))
    nifti_hdr = template_cifti.nifti_header
    del template_cifti
    
    new_img = nib.Cifti2Image(data, header=[ax0, ax1],nifti_header=nifti_hdr)
    new_img.update_headers()

    new_img.to_filename(output_dir)
    

/home/fralberti/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [4]:
### VARIABLES TO SET BEFORE RUNNING
# directory containing subdirectories named fter subject IDs that contain the timeseries and surface files
root_dir = "/home/fralberti/Data/HCP/"
# directory where all intermediate files and the final output will be saved
output_dir = "/home/fralberti/Data/Gradientile/"
# list of IDs of subjects to include in the analyses
f = open(f'{root_dir}subj_IDs.txt', 'r')
subj_id = np.array(f.read().splitlines()[0:51])
del f

In [3]:
### Smooth time series
kernel = 6

print('Smoothing time series\nCurrent subject:')
for i,subj in enumerate(subj_id):
    acquisitions = ['rfMRI_REST1_LR','rfMRI_REST1_RL','rfMRI_REST2_LR','rfMRI_REST2_RL']
    print(f'\t{i+1}/{len(subj_id)}\t{subj}')
    for acq_tmp in acquisitions:
        in_dtseries = f'{root_dir}{subj}/Rest/*{acq_tmp}_Atlas_MSMAll_hp2000_clean.dtseries.nii'
        out_dtseries = f'{root_dir}{subj}/Rest/*{acq_tmp}_Atlas_MSMAll_hp2000_clean_smooth{kernel}.dtseries.nii'
        L_surface = f'{root_dir}{subj}/Structural/{subj}.L.midthickness_MSMAll.32k_fs_LR.surf.gii'
        R_surface = f'{root_dir}{subj}/Structural/{subj}.R.midthickness_MSMAll.32k_fs_LR.surf.gii'
        
        smooth_dtseries(in_dtseries, out_dtseries, kernel, L_surface, R_surface)
print('Done!')

Smoothing time series
Current subject:
	1/51	100206
	2/51	100307
	3/51	100408
	4/51	100610
	5/51	101006
	6/51	101107
	7/51	101309
	8/51	101410



While running:
wb_command -cifti-smoothing '/home/fralberti/Data/HCP/101410/Rest/*rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii' -fwhm 6 6 COLUMN '/home/fralberti/Data/HCP/101410/Rest/*rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean_smooth6.dtseries.nii' -left-surface /home/fralberti/Data/HCP/101410/Structural/101410.L.midthickness_MSMAll.32k_fs_LR.surf.gii -right-surface /home/fralberti/Data/HCP/101410/Structural/101410.R.midthickness_MSMAll.32k_fs_LR.surf.gii

ERROR: failed to open file '/home/fralberti/Data/HCP/101410/Rest/*rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii', file does not exist, or folder permissions prevent seeing it


While running:
wb_command -cifti-smoothing '/home/fralberti/Data/HCP/101410/Rest/*rfMRI_REST2_RL_Atlas_MSMAll_hp2000_clean.dtseries.nii' -fwhm 6 6 COLUMN '/home/fralberti/Data/HCP/101410/Rest/*rfMRI_REST2_RL_Atlas_MSMAll_hp2000_clean_smooth6.dtseries.nii' -left-surface /home/fralberti/Data/HCP/101410/Structural/101410.L.midthickness_MSMAll.32k_

	9/51	101915
	10/51	102008
	11/51	102109
	12/51	102311
	13/51	102513
	14/51	102614
	15/51	102715
	16/51	102816
	17/51	103010
	18/51	103111
	19/51	103212
	20/51	103414
	21/51	103515
	22/51	103818
	23/51	104012
	24/51	104416
	25/51	104820
	26/51	105014
	27/51	105115
	28/51	105216
	29/51	105620
	30/51	105923
	31/51	106016
	32/51	106319
	33/51	106521
	34/51	106824
	35/51	107018
	36/51	107220



While running:
wb_command -cifti-smoothing '/home/fralberti/Data/HCP/107220/Rest/*rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii' -fwhm 6 6 COLUMN '/home/fralberti/Data/HCP/107220/Rest/*rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean_smooth6.dtseries.nii' -left-surface /home/fralberti/Data/HCP/107220/Structural/107220.L.midthickness_MSMAll.32k_fs_LR.surf.gii -right-surface /home/fralberti/Data/HCP/107220/Structural/107220.R.midthickness_MSMAll.32k_fs_LR.surf.gii

ERROR: failed to open file '/home/fralberti/Data/HCP/107220/Rest/*rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii', file does not exist, or folder permissions prevent seeing it


While running:
wb_command -cifti-smoothing '/home/fralberti/Data/HCP/107220/Rest/*rfMRI_REST2_RL_Atlas_MSMAll_hp2000_clean.dtseries.nii' -fwhm 6 6 COLUMN '/home/fralberti/Data/HCP/107220/Rest/*rfMRI_REST2_RL_Atlas_MSMAll_hp2000_clean_smooth6.dtseries.nii' -left-surface /home/fralberti/Data/HCP/107220/Structural/107220.L.midthickness_MSMAll.32k_

	37/51	107321
	38/51	107422
	39/51	107725
	40/51	108020
	41/51	108121
	42/51	108222
	43/51	108323
	44/51	108525
	45/51	108828
	46/51	109123
	47/51	109325
	48/51	109830
	49/51	110007
	50/51	110411
	51/51	110613
Done!


In [4]:
### Normalize and conatenate time series
print('Concatenating smoothed time series\nCurrent subject:')
for subj in subj_id:
    print(f'\t{subj}')
    paths_to_dtseries = glob.glob(f'{root_dir}{subj}/Rest/*REST1*_smooth6.dtseries.nii')
    dtseries_out = concat_dtseries(paths_to_dtseries)
    np.save(f'{root_dir}{subj}/Rest/REST1_full_s6.npy',dtseries_out)
    del dtseries_out
print('Done!')

Concatenating smoothed time series
Current subject:
	100206
	100307
	100408
	100610
	101006
	101107
	101309
	101410
	101915
	102008
	102109
	102311
	102513
	102614
	102715
	102816
	103010
	103111
	103212
	103414
	103515
	103818
	104012
	104416
	104820
	105014
	105115
	105216
	105620
	105923
	106016
	106319
	106521
	106824
	107018
	107220
	107321
	107422
	107725
	108020
	108121
	108222
	108323
	108525
	108828
	109123
	109325
	109830
	110007
	110411
	110613
Done!


In [3]:
### Compute GCCA
batch_size = 3
batch_num = int(len(subj_id)/batch_size)
batches = [np.arange(x,x+batch_size) for x in np.arange(0,len(subj_id),batch_size)]

print("Running GCCA\nCurrent batch:")
for batch_tmp in batches:
    print(f'\t{subj_id[batch_tmp]}')
    
    dtseries_batch = [np.array(np.load(f'{root_dir}{subj}/Rest/REST1_full_s6.npy')).T for subj in subj_id[batch_tmp]]
    res = embed.GCCA(n_components=10)
    res = res.fit_transform(dtseries_batch)
    del dtseries_batch

    for i,subj in enumerate(subj_id[batch_tmp]):
        template_cifti = nib.load(f'{root_dir}{subj}/Structural/{subj}.aparc.32k_fs_LR.dlabel.nii')
        grads = res[i,:,:].T
        mk_grad1_dscalar(grads, template_cifti, f'{root_dir}{subj}/{subj}.REST1_GCCA_s6.32k_fs_LR.dscalar.nii')
        del template_cifti
    del res

print("Done!")

Running GCCA
Current batch:
	['100206' '100307' '100408']


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


	['100610' '101006' '101107']


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


	['101309' '101410' '101915']
	['102008' '102109' '102311']


...

	['108323' '108525' '108828']
	['109123' '109325' '109830']


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


	['110007' '110411' '110613']


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Done!


In [ ]:
### Compute GCCA
batch_size = 2
batch_num = int(len(subj_id)/batch_size)
batches = [np.arange(x,x+batch_size) for x in np.arange(0,len(subj_id),batch_size)]

gcca = embed.GCCA(n_components=3, tall=True) # try save gcca out
print("Running GCCA on batch:")
# for batch_tmp in batches:
for subj in subj_id:
    print(f'\t{subj}')
    Xs = np.load(f'{root_dir}{subj}/Rest/REST1_full_s6.npy').T
    onoff = [True if subj==subj_id[-1] else False][0]
    gcca.partial_fit(Xs, multiview_step=onoff)
    del Xs
    
for i,subj in enumerate(subj_id):
    print(f'\t{subj}')

    X = np.load(f'{root_dir}{subj}/Rest/REST1_full_s6.npy').T
    grads = gcca.transform(X,view_idx=i)
    grads = grads.T
    template_cifti = nib.load(f'{root_dir}{subj}/Structural/{subj}.aparc.32k_fs_LR.dlabel.nii')
    ct.save_dscalar(grads, template_cifti, f'{root_dir}{subj}/{subj}.REST1_GCCA_s6.32k_fs_LR.dscalar.nii')
    
    del template_cifti, grads

print("Done!")

In [5]:
## Invert gradients to allign across subject (if needed)
subj_to_invert = subj_id

for subj in subj_to_invert:
    grads = nib.load(f'{root_dir}{subj}/{subj}.REST1_GCCA_s6.32k_fs_LR.dscalar.nii')
    grads_inv = -1*grads.get_fdata()
    ct.save_dscalar(grads_inv, grads, f'{root_dir}{subj}/{subj}.GCCA_s6.32k_fs_LR.dscalar.nii', names=['Gradient 1', 'Gradient 2', 'Gradient 3'])